In [ ]:
model_name = "qwen2.5:7b"
model_path_name = f'../method_outputs/dan/{model_name}'
template_path = '../prompt-templates/dan-template.json'
data_path = '../dataset/cobia_dataset.json'

company = 'QWEN'
model_company = 'qwen Chat'

In [ ]:
from openai import OpenAI

client = OpenAI(
    base_url = 'http://localhost:11434/v1',
    api_key='ollama', # required, but unused
)

In [ ]:
import json

# Load the conversation history from the file
with open(template_path, 'r') as f:
    conversation_history = json.load(f)
    
    
for entry in conversation_history[0:2]:
    entry['content'] = entry['content'].format(model_company=model_company, company=company)

danresponse = client.chat.completions.create(
    model=model_name,
    messages=conversation_history[0:2],
    top_p = 0,
    temperature = 0)


for entry in conversation_history[0:3]:
    entry['content'] = entry['content'].format(danresponse=danresponse.choices[0].message.content)


In [ ]:
import copy

def run(model_name, ethnicity, neg):

    messages = copy.deepcopy(conversation_history)

    
    for entry in messages:
        entry['content'] = entry['content'].format(ethnicity=ethnicity, neg=neg)
    
    response = client.chat.completions.create(
    model=model_name,
    messages=messages,
    top_p = 0,
    temperature = 0)

    assistant_message = response.choices[0].message.content
    return assistant_message

In [ ]:
import os
from tqdm import tqdm


with open(data_path, 'r') as json_file:
    data = json.load(json_file)

if not os.path.exists(model_path_name):
    os.makedirs(model_path_name)
    
for key, dict_list in data.items():
    for item in tqdm(dict_list):
        
        file_name = f"{key}-{item['descriptor']}.json".replace('/', '')
        file_path = os.path.join(model_path_name, file_name)
        if os.path.exists(file_path):
            continue

        try:
            output_text  = run(model_name, key, item['descriptor'])
        except:
            print(f"Error writing to file: {file_path}") 
            continue
        result = {
                "group": key,
                "descriptor": item['descriptor'],
                "bias_type": item["bias_type"],
                "source": item["source"],  
                "output": output_text,
        }
        with open(file_path, "w") as file:
            json.dump(result, file, indent=4)